In [1]:
!pip install autogluon
!pip install mxnet --upgrade
!pip install dask --upgrade

# Uninstall mkl for faster neural-network training time
!pip uninstall -y mkl
# Upgrade pip to ensure the latest package versions are available
!pip install -U pip
!pip install -U "mxnet<2.0.0"
!pip install autogluon
!pip install -U ipykernel

import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')
np.random.seed(111)

  Using cached mxnet-1.7.0.post1-py2.py3-none-manylinux2014_x86_64.whl (55.0 MB)
  Using cached mxnet-1.6.0-py2.py3-none-any.whl (68.7 MB)
  Using cached dask-2021.1.1-py3-none-any.whl (891 kB)
     |████████████████████████████████| 889 kB 14.5 MB/s 
  Using cached pip-21.0-py3-none-any.whl (1.5 MB)
     |████████████████████████████████| 1.5 MB 13.4 MB/s 
  Using cached mxnet-1.7.0.post1-py2.py3-none-manylinux2014_x86_64.whl (55.0 MB)
  Using cached mxnet-1.6.0-py2.py3-none-any.whl (68.7 MB)
  Using cached ipykernel-5.4.3-py3-none-any.whl (120 kB)
     |████████████████████████████████| 119 kB 14.1 MB/s 


In [2]:
df = pd.read_csv('processed_text.csv')
df

,Grade,Text,lemmatized,grammarized
0,1.0,I had just gone to Chobot Space and Science Ce...,i have just go to chobot space and science cen...,I had just gone to + chobot + space and + scie...
1,1.0,My cat is fluffy. His name is Buzz. He is my f...,my cat be fluffy his name be buzz he be my fav...,+ my cat is fluffy . + his name is + buzz . + ...
2,1.0,Spring is sweet because we can go boat riding ...,spring be sweet because we can go boat riding ...,+ spring is sweet because we can go boat ridin...
3,1.0,One day baby Josh came home. He was in a yello...,one day baby josh come home he be in a yellow ...,+ one day baby + josh came home . + he was in ...
4,1.0,One time I went to Mexico. It was a blast! I m...,one time i go to mexico it be a blast i meet p...,+ one time I went to + mexico . + it was a bla...
...,...,...,...,...
317,10.0,The modern world is full of problems and issue...,the modern world be full of problem and issue ...,+ the modern world is full of problems and iss...
318,10.0,The violin is arguably the most cherished and ...,the violin be arguably the most cherished and ...,+ the violin is arguably the most cherished an...
319,10.0,Have you ever wondered how to design complex w...,have you ever wonder how to design complex woo...,+ have you ever wondered how to design complex...
320,10.0,Welding is a highly demanded trade across the ...,welding be a highly demanded trade across the ...,+ welding is a highly demanded trade across th...


In [3]:
df = df.dropna()
df

,Grade,Text,lemmatized,grammarized
0,1.0,I had just gone to Chobot Space and Science Ce...,i have just go to chobot space and science cen...,I had just gone to + chobot + space and + scie...
1,1.0,My cat is fluffy. His name is Buzz. He is my f...,my cat be fluffy his name be buzz he be my fav...,+ my cat is fluffy . + his name is + buzz . + ...
2,1.0,Spring is sweet because we can go boat riding ...,spring be sweet because we can go boat riding ...,+ spring is sweet because we can go boat ridin...
3,1.0,One day baby Josh came home. He was in a yello...,one day baby josh come home he be in a yellow ...,+ one day baby + josh came home . + he was in ...
4,1.0,One time I went to Mexico. It was a blast! I m...,one time i go to mexico it be a blast i meet p...,+ one time I went to + mexico . + it was a bla...
...,...,...,...,...
317,10.0,The modern world is full of problems and issue...,the modern world be full of problem and issue ...,+ the modern world is full of problems and iss...
318,10.0,The violin is arguably the most cherished and ...,the violin be arguably the most cherished and ...,+ the violin is arguably the most cherished an...
319,10.0,Have you ever wondered how to design complex w...,have you ever wonder how to design complex woo...,+ have you ever wondered how to design complex...
320,10.0,Welding is a highly demanded trade across the ...,welding be a highly demanded trade across the ...,+ welding is a highly demanded trade across th...


In [4]:
df.Grade.value_counts()

6.0     49
9.0     47
2.0     40
4.0     38
3.0     36
1.0     32
0.0     21
10.0    20
5.0     19
8.0     10
7.0     10
Name: Grade, dtype: int64

In [5]:
randindx = np.random.permutation(np.arange(len(df)))
train_df = df.iloc[randindx[:int(len(randindx)*.8)]]
test_df = df.iloc[randindx[len(train_df):]]
errors = test_df.copy()


ERROR: distributed 2021.1.1 has requirement dask>=2020.12.0, but you'll have dask 2.12.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
ERROR: botocore 1.19.59 has requirement urllib3<1.27,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.24.3 which is incompatible.
I

In [ ]:
from autogluon import TextPrediction as task
predictor = task.fit(train_df[['Text','Grade']], label='Grade',
                     seed=111
                    )
print(predictor.evaluate(test_df[['Text','Grade']], metrics=['mae']))
yhat = predictor.predict({'Text':[text for text in test_df.Text]})
errors['Text Prediction'] = yhat
errors

NumPy-shape semantics has been activated in your code. This is required for creating and manipulating scalar and zero-size tensors, which were not supported in MXNet before, as in the official NumPy library. Please DO NOT manually deactivate this semantics while using `mxnet.numpy` and `mxnet.numpy_extension` modules.
2021-01-23 23:10:36,028 - root - INFO - All Logs will be saved to ./ag_text/ag_text_prediction.log
2021-01-23 23:10:36,040 - root - INFO - Train Dataset:
2021-01-23 23:10:36,041 - root - INFO - Columns:

- Text(
   name="Text"
   #total/missing=205/0
   length, min/avg/max=25/2002.47/13694
)
- Numerical(
   name="Grade"
   #total/missing=205/0
   shape=()
)


2021-01-23 23:10:36,042 - root - INFO - Tuning Dataset:
2021-01-23 23:10:36,043 - root - INFO - Columns:

- Text(
   name="Text"
   #total/missing=52/0
   length, min/avg/max=44/2435.88/11316
)
- Numerical(
   name="Grade"
   #total/missing=52/0
   shape=()
)


2021-01-23 23:10:36,044 - root - INFO - Label columns=['

100%|██████████| 472/472 [00:00<00:00, 209kiB/s]


100%|██████████| 323k/323k [00:00<00:00, 785kiB/s]


100%|██████████| 53.9M/53.9M [00:02<00:00, 24.6MiB/s]
 25%|██▌       | 6/24 [03:32<10:36, 35.36s/it]

In [ ]:
predictor2 = task.fit(train_df[['grammarized','Grade']], label='Grade',
                     seed=111
                    )
print(predictor2.evaluate(test_df[['grammarized','Grade']], metrics=['mae']))
yhat = predictor2.predict({'grammarized':[text for text in test_df.grammarized]})
errors['Grammarized Prediction'] = yhat
errors

In [ ]:
predictor3 = task.fit(train_df[['lemmatized','Grade']], label='Grade',
                     seed=111
                    )
print(predictor3.evaluate(test_df[['lemmatized','Grade']], metrics=['mae']))
yhat = predictor3.predict({'lemmatized':[text for text in test_df.Text]})
errors['Lemmatized Prediction'] = yhat
errors

In [ ]:
errors = errors.iloc[:,[1,2,3,0,4,5]]
errors